In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Evaluation Script Template :

This is the template we used for evaluating the models for different experiments. For each experiment, we modify the test dataset and models accordingly

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
!pip install transformers

In [ ]:
!pip install aiohttp

In [ ]:
import json
with open("/kaggle/input/ollama-ranked/ollama/ollama_test_subq_quantemp_bm25.json") as f:
    test_data = json.load(f)

In [ ]:
#For bart based models
# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader, Dataset
# from transformers import AutoTokenizer
# from sklearn.metrics import classification_report

# with open("/kaggle/input/testclaimsnlp/test_claims_quantemp.json") as f:
#     test_data_orig = json.load(f)

# # class MultiClassClassifier(nn.Module):
# #     def __init__(self, bert_model_path, labels_count, hidden_dim=768, mlp_dim=500, extras_dim=100, dropout=0.1, freeze_bert=False):
# #         super().__init__()

# #         self.roberta = AutoModel.from_pretrained(bert_model_path, output_hidden_states=True, output_attentions=True)
# #         self.dropout = nn.Dropout(dropout)
# #         self.mlp = nn.Sequential(
# #             nn.Linear(hidden_dim, mlp_dim),
# #             nn.ReLU(),
# #             nn.Linear(mlp_dim, labels_count)
# #         )
# #         if freeze_bert:
# #             print("Freezing layers")
# #             for param in self.roberta.parameters():
# #                 param.requires_grad = False

# #     def forward(self, tokens, masks):
# #         outputs = self.roberta(tokens, attention_mask=masks)
# #         pooled_output = outputs["last_hidden_state"][:, 0]  # [CLS] token representation
# #         dropout_output = self.dropout(pooled_output)
# #         mlp_output = self.mlp(dropout_output)
# #         return mlp_output
# from torch import nn
# class MultiClassClassifier(nn.Module):
#     def __init__(self, bert_model_path, labels_count, hidden_dim=768, mlp_dim=500, extras_dim=100, dropout=0.1, freeze_bert=False):
#         super().__init__()

#         self.roberta = AutoModel.from_pretrained(bert_model_path,output_hidden_states=True,output_attentions=True)
#         self.dropout = nn.Dropout(dropout)
#         self.mlp = nn.Sequential(
#             nn.Linear(hidden_dim, mlp_dim),
#             nn.ReLU(),
#             nn.Linear(mlp_dim, labels_count)
#         )

#         if freeze_bert:
#             print("Freezing layers")
#             for param in self.roberta.parameters():
#                 param.requires_grad = False

#     def forward(self, tokens, masks):
#         output = self.roberta(tokens, attention_mask=masks)
#         # Extract the last hidden state
#         hidden_states = output.last_hidden_state
#         # Use the hidden state of the first token (usually [CLS])
#         pooled_output = hidden_states[:, 0, :]
#         # Apply dropout
#         pooled_output = self.dropout(pooled_output)
#         # Pass through the MLP
#         mlp_output = self.mlp(pooled_output)
#         return mlp_output



# # Initialize the model
# from transformers import AutoTokenizer,AutoModel

# # Load the BERT tokenizer.
# print('Loading BERT tokenizer...')
# tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli', do_lower_case=True)
# # bert_model_path = '/kaggle/input/mathrobertaquantemp/transformers/trained-large-roberta-mnli/1/model_roberta_large_oracle'
# num_classes = 3  # Set the number of classes based on your task
# model =   MultiClassClassifier('facebook/bart-large-mnli', num_classes, 1024,768,140,dropout=0.1,freeze_bert=False)
# # Load the weights
# checkpoint_path = '/kaggle/input/d/aratrikad/bart-mnli/BART-large-MNLI-trained/model_weights'
# model.load_state_dict(torch.load(checkpoint_path))

# # Ensure the model is on the GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Define a custom dataset class for the test data
# class CustomDataset(Dataset):
#     def __init__(self, data, tokenizer, max_len):
#         self.data = data
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         item = self.data[idx]
#         claim = item['claim']
# #         print("claim")
#         concatenated_evidence = " ".join(item['retrieved_evidence'])
# #         print(concatenated_evidence)
#         label_mapping = {
#                 "True": 2,
#                 "False": 1,
#                 "Conflicting": 0
#             }
#         feature = "[Claim]:"+claim+"[Evidences]:"+concatenated_evidence
        
#         encoding = self.tokenizer.encode_plus(
#             feature,
#             add_special_tokens = True,
#             max_length=self.max_len,
#             padding='max_length',
#             truncation=True,
#             return_attention_mask = True,   # Construct attn. masks.
#             return_tensors = 'pt',
#         )
        
#         return {
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten(),
#             'label': torch.tensor(label_mapping[item['label']] ,dtype=torch.long)
#         }

# # Load the test data
# with open('/kaggle/input/ranked-test-evidence/ranked_evidence_test.json', 'r') as f:
#     test_data = json.load(f)


# # Tokenizer
# # tokenizer = AutoTokenizer.from_pretrained(bert_model_path)
# max_len = 256 # Define the max length according to your needs

# # Prepare the test dataset and dataloader
# test_dataset = CustomDataset(test_data, tokenizer, max_len)
# test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)
# # print(test_dataset)

# # Evaluation function
# def evaluate(model, dataloader, device):
#     model.eval()
#     predictions = []
#     true_labels = []
    
#     with torch.no_grad():
#         for batch in dataloader:
# #             print("Next batch")
           
#             input_ids = batch['input_ids'].to(device)
# #             print('-------------------')
# #             print(len(batch['input_ids']))
            
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['label'].to(device)
#             with torch.no_grad():
#                 outputs = model(input_ids, attention_mask)
# #             print(outputs)
#             _, preds = torch.max(outputs, dim=1)

#             predictions.extend(preds.cpu().numpy())
# #             print(predictions)
#             true_labels.extend(labels.cpu().numpy())
            
# #     print(true_labels)
# #     print(predictions)
#     mismatched_entries = []

#     # Loop through both arrays
#     for i in range(len(true_labels)):
#         if true_labels[i] != predictions[i]:
#             test = test_data[i]
#             entry = test_data_orig[i]
#             entry["doc"] = test["retrieved_evidence"]
#             reverse_label_mapping = {
#                 2: "True",
#                 1:"False",
#                 0:"Conflicting"
#             }
#             entry["predicted"] = reverse_label_mapping[predictions[i]]
#             mismatched_entries.append(entry)
# #             print(entry)
#     mismatched_entries_dict = {}
#     for index, element in enumerate(mismatched_entries):
# #         print(mismatched_entries[index])
#         mismatched_entries_dict[index] = element
    
#     with open('/kaggle/working/qualitative_result_mismatch_mathroberta_mnli_exp1_final.json', 'w') as f:
#         json.dump(mismatched_entries_dict, f)
    
    
#     return true_labels, predictions

# # Evaluate the model
# true_labels, predictions = evaluate(model, test_dataloader, device)


# # Calculate and print classification report
# report = classification_report(true_labels, predictions, target_names=[ 'Conflicting', 'False', 'True'])
# print(report)


In [ ]:
#Roberta
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from sklearn.metrics import classification_report

with open("/kaggle/input/testclaimsnlp/test_claims_quantemp.json") as f:
    test_data_orig = json.load(f)

from torch import nn
class MultiClassClassifier(nn.Module):
    def __init__(self, bert_model_path, labels_count, hidden_dim=768, mlp_dim=500, extras_dim=100, dropout=0.1, freeze_bert=False):
        super().__init__()

        self.roberta = AutoModel.from_pretrained(bert_model_path,output_hidden_states=True,output_attentions=True)
        self.dropout = nn.Dropout(dropout)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, mlp_dim),
            nn.ReLU(),
            # nn.Linear(mlp_dim, mlp_dim),
            # # nn.ReLU(),
            # # nn.Linear(mlp_dim, mlp_dim),
            # nn.ReLU(),
            nn.Linear(mlp_dim, labels_count)
        )
        # self.softmax = nn.LogSoftmax(dim=1)
        if freeze_bert:
            print("Freezing layers")
            for param in self.roberta.parameters():
                param.requires_grad = False

    def forward(self, tokens, masks):
        outputs = self.roberta(tokens, attention_mask=masks)
        # For classification, using the pooler output (or the first token representation)
        # If using the pooler output, make sure your model supports it and it matches hidden_dim
        pooled_output = outputs["last_hidden_state"][:, 0]  # [CLS] token representation
        dropout_output = self.dropout(pooled_output)
        # concat_output = torch.cat((dropout_output, topic_emb), dim=1)
        # concat_output = self.dropout(concat_output)
        mlp_output = self.mlp(dropout_output)
        # proba = self.sigmoid(mlp_output)
        # proba = self.softmax(mlp_output)

        return mlp_output

# Initialize the model
from transformers import AutoTokenizer,AutoModel

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli', do_lower_case=True)
# bert_model_path = '/kaggle/input/mathrobertaquantemp/transformers/trained-large-roberta-mnli/1/model_roberta_large_oracle'
num_classes = 3  # Set the number of classes based on your task
model =   MultiClassClassifier('/kaggle/input/math-roberta/transformers/math-roberta1/1/Math Roberta', num_classes, 1024,768,140,dropout=0.1,freeze_bert=False)
# Load the weights
checkpoint_path = '/kaggle/input/math-roberta-claim-decomp-trained/MathRoberta-MNLI-ClaimDecomp-trained/model_weights'
model.load_state_dict(torch.load(checkpoint_path))

# Ensure the model is on the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a custom dataset class for the test data
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        claim = item['claim']
#         print("claim")
        concatenated_evidence = " ".join(item['retrieved_evidence'])
#         print(concatenated_evidence)
        label_mapping = {
                "True": 2,
                "False": 1,
                "Conflicting": 0
            }
        feature = "[Claim]:"+claim+"[Evidences]:"+concatenated_evidence
        
        encoding = self.tokenizer.encode_plus(
            feature,
            add_special_tokens = True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label_mapping[item['label']] ,dtype=torch.long)
        }

# Load the test data
with open('/kaggle/input/ollama-ranked/ollama/ollama_test_subq_quantemp_bm25.json', 'r') as f:
    test_data = json.load(f)


# Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(bert_model_path)
max_len = 256 # Define the max length according to your needs

# Prepare the test dataset and dataloader
test_dataset = CustomDataset(test_data, tokenizer, max_len)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)
# print(test_dataset)

# Evaluation function
def evaluate(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
#             print("Next batch")
           
            input_ids = batch['input_ids'].to(device)
#             print('-------------------')
#             print(len(batch['input_ids']))
            
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            with torch.no_grad():
                outputs = model(input_ids, attention_mask)
#             print(outputs)
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds.cpu().numpy())
#             print(predictions)
            true_labels.extend(labels.cpu().numpy())
            
#     print(true_labels)
#     print(predictions)
    mismatched_entries = []

    # Loop through both arrays
    for i in range(len(true_labels)):
        if true_labels[i] != predictions[i]:
            test = test_data[i]
            entry = test_data_orig[i]
            entry["doc"] = test["retrieved_evidence"]
            reverse_label_mapping = {
                2: "True",
                1:"False",
                0:"Conflicting"
            }
            entry["predicted"] = reverse_label_mapping[predictions[i]]
            mismatched_entries.append(entry)
#             print(entry)
    mismatched_entries_dict = {}
    for index, element in enumerate(mismatched_entries):
#         print(mismatched_entries[index])
        mismatched_entries_dict[index] = element
    
    with open('/kaggle/working/qualitative_result_mismatch_mathroberta_ollama_mnli_exp1_final.json', 'w') as f:
        json.dump(mismatched_entries_dict, f)
    
    
    return true_labels, predictions

# Evaluate the model
true_labels, predictions = evaluate(model, test_dataloader, device)


# Calculate and print classification report
report = classification_report(true_labels, predictions, target_names=[ 'Conflicting', 'False', 'True'])
print(report)


In [ ]:
from sklearn.metrics import f1_score
macro_f1 = f1_score(true_labels, predictions, average='macro')
w_f1 =  f1_score(true_labels, predictions, average='weighted')
print(f'M-F1: {macro_f1} ')
print(f'W-F1: {w_f1} ')